In [1]:
#imports
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#parser for twitchtracker
url = f"https://twitchtracker.com/shroud/games"
def parse_games(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Platform; Security; OS-or-CPU; Localization; rv:1.4) Gecko/20030624 Netscape/7.1 (ax)'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    df=pd.read_html(str(soup))[0]
    return df

In [3]:
#returns a df 
first_df = parse_games(url)

#                object
Game             object
Avg. viewers      int64
Max. viewers      int64
Followers        object
Total airtime    object
Last seen        object
dtype: object

In [ ]:
#notes from the discussion with the team on tuesday weak 1

#get df for every broadcast 
#cut out all the games 0% airtime
#take off % @airtime 
#booleanmask on airtime

#can we downcast a string? to respect an efficient df
#in the end safe in dict key=user, value=df
#downcast # column to number.
#store it into csv in the end


In [6]:
#parse the games of streamer
def parse_games_2(user_name):
    user_name.lower()
    headers = {'User-Agent': 'Mozilla/5.0 (Platform; Security; OS-or-CPU; Localization; rv:1.4) Gecko/20030624 Netscape/7.1 (ax)'}
    response = requests.get(url = f"https://twitchtracker.com/{user_name}/games", headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    df=pd.read_html(str(soup))[0]
    return df

In [7]:
Quackity = parse_games_2("Quackity")

In [8]:
#cut out all the games 0% airtime
Quackity.dtypes

#                object
Game             object
Avg. viewers      int64
Max. viewers      int64
Followers        object
Total airtime    object
Last seen        object
dtype: object

In [ ]:
Quackity.head()

In [37]:
Quackity["Total airtime"] = Quackity["Total airtime"].astype(str).str[:-1]

In [44]:
#rename totalairtime to make it more handsome to work
Quackity["totalairtime"] = Quackity["Total airtime"]

In [48]:
Quackity = Quackity.drop(labels="Total airtime", axis=1)

In [ ]:
Quackity

In [50]:
#make to columns out of 1
#see totalairtime. more handsome.

Quackity[["absolute total airtime", "percentage of total airtime"]] = Quackity.totalairtime.str.split(" ", expand=True)

In [52]:
Quackity = Quackity.drop("totalairtime", axis=1)

In [55]:
Quackity

,#,Game,Avg. viewers,Max. viewers,Followers,Last seen,absolute total airtime,percentage of total airtime
0,#1,Minecraft,31406,381550,2570.2 /hour,2021-05-29 21:40:00,13261,34.1
1,#2,Just Chatting,18207,230817,1555.4 /hour,2021-04-23 01:30:00,9447,24.3
2,#3,Toontown Online,1524,4299,236 /hour,2020-06-19 07:20:00,2032,5.2
3,#4,Jackbox Party Packs,126620,232299,5230.6 /hour,2021-04-15 03:40:00,1376,3.5
4,#5,Among Us,20987,109720,2384.7 /hour,2020-12-30 04:30:00,1254,3.2
5,#6,Roblox,76616,202526,4208.7 /hour,2021-03-22 00:10:00,1223,3.1
6,#7,Habbo,18714,117637,1244.9 /hour,2021-01-17 00:20:00,1048,2.7
7,#8,Animal Crossing: New Horizons,3412,4208,557.2 /hour,2020-04-20 04:40:00,942,2.4
8,#9,The Sims 4,19403,164457,1014.3 /hour,2021-03-27 02:30:00,917,2.4
9,#10,Outlast,31344,97536,1883.2 /hour,2020-12-29 06:10:00,823,2.1


In [60]:
Quackity[20:21]

,#,Game,Avg. viewers,Max. viewers,Followers,Last seen,absolute total airtime,percentage of total airtime
20,#21,Moshi Monsters,2455,3048,169.1 /hour,2019-11-21 23:20:00,193,0.5


# Playing around with some scrapping

In [ ]:
pages = range(2,25)
url = f"https://twitchtracker.com/channels/viewership?page=1"
def parse_html(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Platform; Security; OS-or-CPU; Localization; rv:1.4) Gecko/20030624 Netscape/7.1 (ax)'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    df=pd.read_html(str(soup), flavor='bs4')[0]
    return df

In [ ]:
import time
t=1

df = parse_html(url)
for page in pages:
    print(page)
    new_df = parse_html(f"https://twitchtracker.com/channels/viewership?page={page}")
    df = df.append(new_df, ignore_index=True)
    time.sleep(t)
    

In [1]:
df = df.rename(columns={'Unnamed: 2': 'username'})

NameError: name 'df' is not defined

In [ ]:
df = df.rename(columns={'Unnamed: 0': 'id'})

In [ ]:
to_remove = df["id"].str.contains('ads')

In [ ]:
# get all streamer with usernames in a list to feed into scraping

usernames = []
for entry in df['username']:
    if '.ads' not in entry:
        usernames.append(entry)

In [ ]:
# Trying to get link to other social media plattforms from respective twitch/about page of users
# here I chose one of my favorite streamer

import requests
from bs4 import BeautifulSoup

URL= "https://www.twitch.tv/noway4u_sir/about"

headers = {'User-Agent': 'Mozilla/5.0 (Platform; Security; OS-or-CPU; Localization; rv:1.4) Gecko/20030624 Netscape/7.1 (ax)'}
response = requests.get(URL, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")
# response = requests.get(html)
# test = soup.find_all(href=True) # should return all href tags of that page but somehow no social media plattform is shown

# handles = [a["href"] for a in soup.find_all("a", href=True) if("twitter" in a["href"])]
handles2= [a["href"] for a in soup.find_all("a", href=True)]

## other stuff that I tried based on google and our lecture on the API

# links_with_text = []
# for a in soup.find_all('a', href=True): 
#     if a.text: 
#         links_with_text.append(a['href'])
        


# soup = BeautifulSoup(response.content, "html.parser")
# social = soup.find("div", class_="sc-AxjAm jMKFBR")
# # twitter = social.find("div", class_="sc-AxjAm jgNmMs social-media-link__container").find("a").string
# youtube = int(movie.find("span", class_="runtime").string.strip(' min'))
# instagram =  movie.find("h3").find("a").string


# movies.append({'title': title, 'duration': duration})


# movies = []
# for movie in soup.find_all("div", class_="lister-item-content"):


# print(movies[0:2])




# def parse_streamer(streamer):
#     twitter = article.find('p', class_= 'recipe-name').string.strip()
#     difficulty = article.find('span', class_= 'recipe-difficulty').string.strip()
#     prep_time = article.find('span', class_= 'recipe-cooktime').string.strip()
#     return {'name': name, 'difficulty': difficulty, 'prep_time': prep_time}

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
driver = webdriver.Chrome()
URL= "https://www.twitch.tv/noway4u_sir/about"
driver.get(URL)
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()
print(soup.prettify())

In [ ]:
soup.find_all('a', class_='tw-link')

In [ ]:
soup.find_all

### playing around with twitchtracker and get informations

In [2]:
params = {
    "page": 1
}

r = 'https://twitchtracker.com/channels/viewership'
response = requests.get(r,
                       headers={'User-Agent': 'Mozilla/5.0 (Platform; Security; OS-or-CPU; Localization; rv:1.4) Gecko/20030624 Netscape/7.1 (ax)'},
                        params=params
                       )
response

NameError: name 'requests' is not defined

In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')

user_name = []
for user_name in soup.find_all("div", class_="table-responsive"):
    user = user_name.find('table').find('a')
    user_name.append(user)

In [ ]:
#<td><a href="/auronplay">auronplay</a></td>
soup = str(soup)

In [ ]:
read_ = pd.read_html(soup)[0]
read_

In [ ]:
select_names = str(soup.select_one('#channels'))
pd.read_html(select_names)[0]

def parser(start, stop):
    start = int(start)
    stop = int(stop)
    
    pass

In [ ]:
def test_function(start, stop):
    
    for parser in range(start,stop):
        print(parser)
    print("done")



In [ ]:
def parser(stop):
    while int(_) < stop:
        params = {
            "page": _+1
        }

        r = 'https://twitchtracker.com/channels/viewership'
        response = requests.get(r,
                       headers={'User-Agent': 'Mozilla/5.0 (Platform; Security; OS-or-CPU; Localization; rv:1.4) Gecko/20030624 Netscape/7.1 (ax)'},
                        params=params
                       )
        response
        soup = BeautifulSoup(response.content, 'html.parser')
        soup = str(soup)
        read_html = pd.read_html(soup)[0]
        return read_html

In [ ]:
params = range(1,25)
url = f"https://twitchtracker.com/channels/viewership?page={params}"
def parse_html(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Platform; Security; OS-or-CPU; Localization; rv:1.4) Gecko/20030624 Netscape/7.1 (ax)'}
    response = requests.get(url,
                        headers=headers,
                        params=params
                       )
    soup = BeautifulSoup(response.content, 'html.parser')
    soup=str(soup)
    read_ = pd.read_html(soup)
    select_names = str(soup.select_one('#channels'))
    df=pd.DataFrame(read_)
    return df

In [ ]:

def parse_html(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Platform; Security; OS-or-CPU; Localization; rv:1.4) Gecko/20030624 Netscape/7.1 (ax)'}
    result = []
    for params in range(1,25):
        response = requests.get(
            url,
            headers=headers,
            params=params
        )
        soup = BeautifulSoup(response.content, 'html.parser')
        soup = str(soup)
        read_html = pd.read_html(soup)[0]
        result.append(read_html)
    return result

In [ ]:

    
def get_first_page():
    for _ in range(1,25):
        params = _
    params = params

    r = 'https://twitchtracker.com/channels/viewership'
    response = requests.get(r,
                       headers={'User-Agent': 'Mozilla/5.0 (Platform; Security; OS-or-CPU; Localization; rv:1.4) Gecko/20030624 Netscape/7.1 (ax)'},
                        params=str(params)
                       )
    soup = BeautifulSoup(response.content, 'html.parser')
    soup = str(soup)
    print("Hey")
    read = pd.read_html(soup)[0]
    return read